# Quackling — Hybrid Pipeline

Note: this example requires a Milvus instance with hybrid retrieval support.

In [1]:
# requirements for this example:
%pip install -q \
    quackling \
    python-dotenv \
    llama-index-embeddings-huggingface \
    llama-index-llms-huggingface-api \
    flagembedding \
    llama-index-vector-stores-milvus \
    llama-index-postprocessor-flag-embedding-reranker

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from dotenv import load_dotenv
from pydantic import TypeAdapter
from rich.pretty import pprint

load_dotenv()

FILE_PATHS = [
    # "/path/to/local/pdf",  # file path
    "https://arxiv.org/pdf/2206.01062",  # URL (DocLayNet paper)
]
TEXT_QA_TEMPLATE_STR = "Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer:\n"
QUERY = "What is this paper about?"
TOP_K = 5

INGEST = TypeAdapter(bool).validate_python(os.environ.get("INGEST", "True"))

In [3]:
import warnings

warnings.filterwarnings(
    action="ignore", category=UserWarning, module="torchvision|torch"
)

## Setup

### Reader

In [4]:
from quackling.llama_index.readers.docling_reader import DoclingReader

reader = DoclingReader(parse_type=DoclingReader.ParseType.JSON)
load_kwargs = dict(
    file_path=FILE_PATHS,
)

### Node parser

In [5]:
from quackling.llama_index.node_parsers.hier_node_parser import HierarchicalNodeParser

node_parser = HierarchicalNodeParser()

### Dense Embedding model

In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

HF_EMBED_MODEL_ID = "BAAI/bge-m3"

embed_model = HuggingFaceEmbedding(model_name=HF_EMBED_MODEL_ID)

/Users/pva/work/github.com/DS4SD/quackling/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### LLM

In [7]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

HF_API_KEY = os.environ.get("HF_API_KEY")
HF_LLM_MODEL_ID = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm = HuggingFaceInferenceAPI(
    token=HF_API_KEY,
    model_name=HF_LLM_MODEL_ID,
)

/Users/pva/work/github.com/DS4SD/quackling/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


### Vector store

In [8]:
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.vector_stores.milvus.utils import BGEM3SparseEmbeddingFunction

MILVUS_URL = os.environ["MILVUS_URL"]
MILVUS_COLL_NAME = os.environ.get("MILVUS_COLL_NAME", "quackling_hybrid_pipeline")
MILVUS_KWARGS = TypeAdapter(dict).validate_json(os.environ.get("MILVUS_KWARGS", "{}"))
MILVUS_HYBRID_RANKER = os.environ.get("MILVUS_HYBRID_RANKER", "RRFRanker")
MILVUS_HYBRID_RNKR_PARAMS = TypeAdapter(dict).validate_json(
    os.environ.get("MILVUS_HYBRID_RNKR_PARAMS", '{"k": 60}')
)

sparse_embedding = BGEM3SparseEmbeddingFunction()

vector_store = MilvusVectorStore(
    uri=MILVUS_URL,
    dim=len(embed_model.get_text_embedding("hi")),
    collection_name=MILVUS_COLL_NAME,
    overwrite=INGEST,  # not showing follow-up/incremental ingestions in this example
    enable_sparse=True,
    sparse_embedding_function=sparse_embedding,
    hybrid_ranker=MILVUS_HYBRID_RANKER,
    hybrid_ranker_params=MILVUS_HYBRID_RNKR_PARAMS,
    **MILVUS_KWARGS,
)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [9]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode

vector_store_query_mode = VectorStoreQueryMode.HYBRID

### Reranker

In [10]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(model="BAAI/bge-reranker-v2-m3", top_n=TOP_K)

## Preparing vector store index

In [11]:
from llama_index.core import StorageContext, VectorStoreIndex

if INGEST:
    # in this case we ingest the data into the vector store
    docs = reader.load_data(**load_kwargs)
    pprint(docs, max_length=1, max_string=250, max_depth=4)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
        documents=docs,
        embed_model=embed_model,
        storage_context=storage_context,
        transformations=[
            node_parser,
        ],
    )
else:
    # in this case we just load the vector store index
    index = VectorStoreIndex.from_vector_store(vector_store)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

[
│   Document(
│   │   id_='5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc',
│   │   embedding=None,
│   │   metadata={'dl_doc_hash': '5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc'},
│   │   excluded_embed_metadata_keys=['dl_doc_hash'],
│   │   excluded_llm_metadata_keys=['dl_doc_hash'],
│   │   relationships={},
│   │   text='{"_name":"","type":"pdf-document","description":{"title":null,"abstract":null,"authors":null,"affiliations":null,"subjects":null,"keywords":null,"publication_date":null,"languages":null,"license":null,"publishers":null,"url_refs":null,"references":nu'+171180,
│   │   mimetype='text/plain',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   )
]

## RAG

To use the reranker in RAG, we include it in the `node_postprocessors`:

In [12]:
from llama_index.core import PromptTemplate

query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=TOP_K,
    node_postprocessors=[reranker],  # <==
    text_qa_template=PromptTemplate(TEXT_QA_TEMPLATE_STR),
    vector_store_query_mode=vector_store_query_mode,
)
query_res = query_engine.query(QUERY)
pprint(query_res, max_length=1, max_string=250, max_depth=4)

Response(
│   response='This paper presents the DocLayNet dataset, which is a new and challenging dataset for the document conversion and layout analysis research community. The dataset was created by human annotation to obtain reliable layout ground-truth on a wide variety'+313,
│   source_nodes=[
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='41f13be2-80d4-4480-a61b-4f9ce6d850cc',
│   │   │   │   embedding=None,
│   │   │   │   metadata={...},
│   │   │   │   excluded_embed_metadata_keys=[...],
│   │   │   │   excluded_llm_metadata_keys=[...],
│   │   │   │   relationships={...},
│   │   │   │   text='6 CONCLUSION\nIn this paper, we presented the DocLayNet dataset. It provides the document conversion and layout analysis research community a new and challenging dataset to improve and fine-tune novel ML methods on. In contrast to many other datasets,'+270,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=None,
│   │   │   │   end_char_idx=None,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=-1.5540306568145752
│   │   ),
│   │   ... +4
│   ],
│   metadata={'41f13be2-80d4-4480-a61b-4f9ce6d850cc': {'path': '$.main-text[115]'}, ... +4}
)

## Retrieval

In [13]:
retriever = index.as_retriever(
    similarity_top_k=TOP_K,
    vector_store_query_mode=vector_store_query_mode,
)
retr_res = retriever.retrieve(QUERY)
pprint(retr_res, max_length=1, max_string=250, max_depth=4)

[
│   NodeWithScore(
│   │   node=TextNode(
│   │   │   id_='a2b5a711-7159-460a-8460-ddd080b4db77',
│   │   │   embedding=None,
│   │   │   metadata={'path': '$.main-text[88]'},
│   │   │   excluded_embed_metadata_keys=['path'],
│   │   │   excluded_llm_metadata_keys=['path'],
│   │   │   relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...), ... +2},
│   │   │   text='5 EXPERIMENTS\npaper and leave the detailed evaluation of more recent methods mentioned in Section 2 for future work.',
│   │   │   mimetype='text/plain',
│   │   │   start_char_idx=None,
│   │   │   end_char_idx=None,
│   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   metadata_template='{key}: {value}',
│   │   │   metadata_seperator='\n'
│   │   ),
│   │   score=0.032786883413791656
│   ),
│   ... +4
]

Here are the retrieval results (basic fields) prior to reranking:

In [14]:
before_rrk = [
    dict(path=node.metadata["path"], text=node.text, score=node.score)
    for node in retr_res
]
pprint(before_rrk, max_string=250)

[
│   {
│   │   'path': '$.main-text[88]',
│   │   'text': '5 EXPERIMENTS\npaper and leave the detailed evaluation of more recent methods mentioned in Section 2 for future work.',
│   │   'score': 0.032786883413791656
│   },
│   {
│   │   'path': '$.main-text[115]',
│   │   'text': '6 CONCLUSION\nIn this paper, we presented the DocLayNet dataset. It provides the document conversion and layout analysis research community a new and challenging dataset to improve and fine-tune novel ML methods on. In contrast to many other datasets,'+270,
│   │   'score': 0.0320020467042923
│   },
│   {
│   │   'path': '$.main-text[74]',
│   │   'text': '4 ANNOTATION CAMPAIGN\nThe complete annotation guideline is over 100 pages long and a detailed description is obviously out of scope for this paper. Nevertheless, it will be made publicly available alongside with DocLayNet for future reference.',
│   │   'score': 0.016129031777381897
│   },
│   {
│   │   'path': '$.main-text[10]',
│   │   'text': 'CCS CONCEPTS\nPermission to make digital or hard copies of part or all of this work for personal or classroom use is granted without fee provided that copies are not made or distributed for profit or commercial advantage and that copies bear this noti'+160,
│   │   'score': 0.01587301678955555
│   },
│   {
│   │   'path': '$.main-text[11]',
│   │   'text': "CCS CONCEPTS\nKDD '22, August 14-18, 2022, Washington, DC, USA © 2022 Copyright held by the owner/author(s). ACM ISBN 978-1-4503-9385-0/22/08. https://doi.org/10.1145/3534678.3539043",
│   │   'score': 0.015625
│   }
]

Here we apply the reranker on the retrieved results — as you can see, the reranker has indeed changed the order:

In [15]:
reranked = reranker.postprocess_nodes(nodes=retr_res, query_str=QUERY)
after_rrk = [
    dict(path=node.metadata["path"], text=node.text, score=node.score)
    for node in reranked
]
pprint(after_rrk, max_string=250)

[
│   {
│   │   'path': '$.main-text[115]',
│   │   'text': '6 CONCLUSION\nIn this paper, we presented the DocLayNet dataset. It provides the document conversion and layout analysis research community a new and challenging dataset to improve and fine-tune novel ML methods on. In contrast to many other datasets,'+270,
│   │   'score': -1.5540306568145752
│   },
│   {
│   │   'path': '$.main-text[74]',
│   │   'text': '4 ANNOTATION CAMPAIGN\nThe complete annotation guideline is over 100 pages long and a detailed description is obviously out of scope for this paper. Nevertheless, it will be made publicly available alongside with DocLayNet for future reference.',
│   │   'score': -4.99463415145874
│   },
│   {
│   │   'path': '$.main-text[88]',
│   │   'text': '5 EXPERIMENTS\npaper and leave the detailed evaluation of more recent methods mentioned in Section 2 for future work.',
│   │   'score': -6.130667686462402
│   },
│   {
│   │   'path': '$.main-text[10]',
│   │   'text': 'CCS CONCEPTS\nPermission to make digital or hard copies of part or all of this work for personal or classroom use is granted without fee provided that copies are not made or distributed for profit or commercial advantage and that copies bear this noti'+160,
│   │   'score': -6.377060890197754
│   },
│   {
│   │   'path': '$.main-text[11]',
│   │   'text': "CCS CONCEPTS\nKDD '22, August 14-18, 2022, Washington, DC, USA © 2022 Copyright held by the owner/author(s). ACM ISBN 978-1-4503-9385-0/22/08. https://doi.org/10.1145/3534678.3539043",
│   │   'score': -7.342046737670898
│   }
]